In [20]:
# default_exp visualization

%reload_ext autoreload
%autoreload 2

# gen_chart_data

In [163]:
# export
def gen_chart_data(data):
    """
    把profile数据转换为绘图数据
    """

    

    df = data.copy()

    df['date'] = df.index.map(str).str.split().str.get(0)

    chart_data = {}

    chart_data['categoryData'] = df['date'].tolist()

    chart_data['values'] = df['date, open, close, low, high, volume'.split(', ')].values.tolist()

    df['flag'] = 1
    
    df.loc[df.close>df.open, 'flag'] = -1
#     print(df.flag.value_counts())
    _tmp = df[['volume', 'flag']].reset_index(drop=True)

    chart_data['volumes'] = _tmp.reset_index().values.tolist()
    
    return chart_data

In [196]:
# export
def calculate_ma(day_count: int, data):
    result: List[Union[float, str]] = []
    for i in range(len(data["values"])):
        if i < day_count:
            result.append("-")
            continue
        sum_total = 0.0
        for j in range(day_count):
            sum_total += float(data["values"][i - j][1])
        result.append(abs(float("%.3f" % (sum_total / day_count))))
    return result


def draw_charts(chart_data, name='', markpoints1=[], markpoints2=[]):
    """
    
    """
    from pyecharts import options as opts
    from pyecharts.charts import Kline, Line, Bar, Grid
    from typing import List, Union
    if markpoints1:
        markpoints1 = [opts.MarkPointItem(
                        coord=(chart_data["categoryData"][i], chart_data['values'][i][2]), 
                        value=round(v, 2),
#                     name="t", 
                    symbol='pin',
                    symbol_size=(20,45)
            
                ) for i, v in markpoints1]
    if markpoints2:
        markpoints2 = [opts.MarkPointItem(
                        coord=(chart_data["categoryData"][i], chart_data['values'][i][2]), 
                        value=round(v, 2),
#                     name="t", 
                    itemstyle_opts=opts.ItemStyleOpts(
                        color="#00da3c"
                    ),
                    symbol='arrow',
                    symbol_size=(10,45)
            
                ) for i, v in markpoints2]
    markpoints = markpoints1 + markpoints2
    kline_data = [data[1:-1] for data in chart_data["values"]]  # open, close, low, high
    kline = (
        Kline()
        .add_xaxis(xaxis_data=chart_data["categoryData"])
        .add_yaxis(
            series_name=name,
            y_axis=kline_data,
            itemstyle_opts=opts.ItemStyleOpts(color="#ec0000", color0="#00da3c"),
            markline_opts=opts.MarkLineOpts(
                data=[
                    opts.MarkLineItem(type_="max", value_dim="close"),  
                    opts.MarkLineItem(type_="min", value_dim="close")
                     ]
            ),
            markpoint_opts=opts.MarkPointOpts(
                data=markpoints
#                 [
#                     opts.MarkPointItem(type_="max", name="最大值"),
#                     opts.MarkPointItem(type_="min", name="最小值"),
#                     opts.MarkPointItem(
#                         coord=(chart_data["categoryData"][df.shape[0]-100], df.close.iloc[-100]), 
#                         value=df.close.iloc[-100],
#                         name="t"
#                     ),
#                 ]
            ),
        )
        .set_global_opts(
            legend_opts=opts.LegendOpts(
                is_show=False, pos_bottom=10, pos_left="center"
            ),
            datazoom_opts=[
#                 opts.DataZoomOpts(
#                     is_show=False,
# #                     type_="inside",
#                     xaxis_index=[0, 1],
#                     range_start=98,
#                     range_end=100,
#                 ),
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=[0, 1],
                    type_="slider",
                    pos_top="85%",
                    range_start=98,
                    range_end=100,
                ),
            ],
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitarea_opts=opts.SplitAreaOpts(
                    is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                ),
            ),
            tooltip_opts=opts.TooltipOpts(
                trigger="axis",
                axis_pointer_type="cross",
                background_color="rgba(245, 245, 245, 0.8)",
                border_width=1,
                border_color="#ccc",
                textstyle_opts=opts.TextStyleOpts(color="#000"),
            ),
            visualmap_opts=opts.VisualMapOpts(
                is_show=False,
                dimension=2,
                series_index=5,
                is_piecewise=True,
                pieces=[
                    {"value": 1, "color": "#00da3c"},
                    {"value": -1, "color": "#ec0000"},
                ],
            ),
            axispointer_opts=opts.AxisPointerOpts(
                is_show=True,
                link=[{"xAxisIndex": "all"}],
                label=opts.LabelOpts(background_color="#777"),
            ),
            brush_opts=opts.BrushOpts(
                x_axis_index="all",
                brush_link="all",
                out_of_brush={"colorAlpha": 0.1},
                brush_type="lineX",
            ),
            toolbox_opts=opts.ToolboxOpts(is_show=True),
        )
    )

    line = (
        Line()
        .add_xaxis(xaxis_data=chart_data["categoryData"])
        .add_yaxis(
            series_name="MA5",
            y_axis=calculate_ma(day_count=5, data=chart_data),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA10",
            y_axis=calculate_ma(day_count=10, data=chart_data),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA20",
            y_axis=calculate_ma(day_count=20, data=chart_data),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA30",
            y_axis=calculate_ma(day_count=30, data=chart_data),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
    )

    bar = (
        Bar()
        .add_xaxis(xaxis_data=chart_data["categoryData"])
        .add_yaxis(
            series_name="Volume",
            y_axis=chart_data["volumes"],
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category",
                is_scale=True,
                grid_index=1,
                boundary_gap=False,
                axisline_opts=opts.AxisLineOpts(is_on_zero=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False),
                split_number=20,
                min_="dataMin",
                max_="dataMax",
            ),
            yaxis_opts=opts.AxisOpts(
                grid_index=1,
                is_scale=True,
                split_number=2,
                axislabel_opts=opts.LabelOpts(is_show=False),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )

    # Kline And Line
    overlap_kline_line = kline.overlap(line)

    # Grid Overlap + Bar
    grid_chart = Grid(
        init_opts=opts.InitOpts(
            width="1000px",
            height="600px",
            animation_opts=opts.AnimationOpts(animation=False),
        )
    )
    grid_chart.add(
        overlap_kline_line,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", height="50%"),
    )
    grid_chart.add(
        bar,
        grid_opts=opts.GridOpts(
            pos_left="10%", pos_right="8%", pos_top="63%", height="16%"
        ),
    )

    return grid_chart.render_notebook()

# test

In [21]:
from code.datasource import SinaDataSource

In [129]:
ds = SinaDataSource('大盘', '20100101')

3851
code:sh000001, start_date: 20100101, end_date: 20200718, datalen: 3851
               open     high      low    close       volume
day                                                        
2010-01-04  3289.75  3295.28  3243.32  3243.76  10944792800
2010-01-05  3254.47  3290.51  3221.46  3282.18  12611506400
               open     high      low    close       volume
day                                                        
2020-07-16  3356.36  3373.53  3209.73   3210.1  49061312300
2020-07-17   3214.4  3252.78  3181.28  3214.13  35965241300


In [130]:
ds.data.shape

(2561, 5)

In [195]:
draw_charts(gen_chart_data(ds.data), [-1, -20])

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/pyecharts/charts/composite_charts/grid.py:17: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


# nbexport

In [3]:
from nbdev.export import *
notebook2script('visualization.ipynb')

Converted visualization.ipynb.
